## Basic Agent

In [14]:
!pip install openai tavily-python

In [15]:
from google.colab import userdata
import os

In [16]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [17]:
from openai import OpenAI

In [18]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [19]:
from tavily import TavilyClient

def tavily_deep_search(query, search_depth="advanced", max_results=5):
    """
    Perform a deep search using Tavily API.

    Args:
        query: The search query
        search_depth: 'basic' or 'advanced' - advanced provides more comprehensive results
        max_results: Number of results to return (1-20)

    Returns:
        dict: Search results with sources, content, and citations
    """
    try:
        # Initialize Tavily client
        tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

        # Perform the search
        response = tavily_client.search(
            query=query,
            search_depth=search_depth,
            max_results=max_results,
            include_answer=True,  # Get AI-generated answer
            include_raw_content=False,  # Set to True if you need full page content
            include_images=False  # Set to True if you need images
        )

        # Format the results
        formatted_results = {
            "query": query,
            "answer": response.get("answer", ""),
            "results": [],
            "search_depth": search_depth
        }

        # Extract relevant information from each result
        for result in response.get("results", []):
            formatted_results["results"].append({
                "title": result.get("title", ""),
                "url": result.get("url", ""),
                "content": result.get("content", ""),
                "score": result.get("score", 0),  # Relevance score
            })

        return formatted_results

    except Exception as e:
        return {
            "query": query,
            "error": str(e),
            "results": []
        }

In [20]:
search_results = tavily_deep_search("What are the latest breakthroughs in quantum computing and their practical applications?")

In [21]:
search_results

{'query': 'What are the latest breakthroughs in quantum computing and their practical applications?',
 'answer': 'Quantum computing breakthroughs include improved drug discovery and stronger cybersecurity, with practical applications in healthcare and data protection. Google and IBM are leading in quantum research, aiming for scalable, fault-tolerant quantum computers. Quantum computing is expected to revolutionize industries and daily life.',
 'results': [{'title': 'Quantum Breakthroughs: NIST & SQMS Lead the Way',
   'url': 'https://www.nist.gov/news-events/news/2025/04/quantum-breakthroughs-nist-sqms-lead-way',
   'content': 'Quantum computing may seem abstract, but its real-world implications are practical and widespread. The quantum research being done will lead to better healthcare due to quantum computers accelerating drug discovery and other medical research. Stronger cyber security measures and encryption methods protecting sensitive data will become available as quantum compu

In [26]:
# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "tavily_deep_search",
        "description": "Perform a deep web search using Tavily to get comprehensive, well-researched information with citations from multiple authoritative sources.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to research on the web",
                },
                "search_depth": {
                    "type": "string",
                    "description": "The depth of the search. 'basic' for quick results, 'advanced' for comprehensive research with more sources",
                    "enum": ["basic", "advanced"]
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of search results to return (1-20)",
                    "minimum": 1,
                    "maximum": 20
                }
            },
            "required": ["query"],  # Only query is required
        },
    },
]

In [22]:
import json
def agent(input_list : list):
  response = client.responses.create(
      model="gpt-4.1-mini",
      tools=tools,
      input=input_list,
  )
  print("Initial response:")
  print(json.dumps(response.model_dump(), indent=2))
  # Save function call outputs for subsequent requests
  input_list += response.output


  for item in response.output:
    if item.type == "function_call":
        if item.name == "tavily_deep_search":
            # 3. Execute the function logic for tavily_deep_search
            args = json.loads(item.arguments)
            search_results = tavily_deep_search(
                query=args["query"],
                search_depth=args.get("search_depth", "advanced"),
                max_results=args.get("max_results", 5)
            )

            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps(search_results)
            })

  print("Final input:")
  print(input_list)

  response = client.responses.create(
      model="gpt-4.1-mini",
      instructions="Respond with a comprehensive answer based on the web search results. Include citations to the sources used.",
      tools=tools,
      input=input_list,
  )
  return response


In [24]:
input_list = [
    {"role": "user", "content": "What are the latest breakthroughs in Artificial General Intelligence?"}
]

In [27]:
agent_response = agent(input_list)

Initial response:
{
  "id": "resp_02696a4eb84fedf80068ea079c7d4081948ac8d50bd2d4f729",
  "created_at": 1760167836.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "response",
  "output": [
    {
      "arguments": "{\"query\":\"latest breakthroughs in Artificial General Intelligence\",\"search_depth\":\"advanced\",\"max_results\":10}",
      "call_id": "call_MPSYVlNSmmlVpgin2Z6uv1Ou",
      "name": "tavily_deep_search",
      "type": "function_call",
      "id": "fc_02696a4eb84fedf80068ea079d56608194a0dbb4be73f39dcd",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "tavily_deep_search",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "The search query to research on the web"
          },
      

In [28]:
print("\n" + "="*80)
print("Final output:")
print("="*80)
print(agent_response.model_dump_json(indent=2))
print("\n" + "="*80)
print("Assistant Response:")
print("="*80)
print(agent_response.output_text)


Final output:
{
  "id": "resp_02696a4eb84fedf80068ea07a33f4c8194a63027f78c4be003",
  "created_at": 1760167843.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond with a comprehensive answer based on the web search results. Include citations to the sources used.",
  "metadata": {},
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "response",
  "output": [
    {
      "id": "msg_02696a4eb84fedf80068ea07a48c888194b8d6e3d2138bd68e",
      "content": [
        {
          "annotations": [],
          "text": "The latest breakthroughs in Artificial General Intelligence (AGI) as of 2025 include several key advances in AI models, sensory integration, reasoning, and real-world applications:\n\n1. **Advanced Large Language Models**: Leading AI systems like OpenAI’s GPT-5 and DeepMind’s Gemini have pushed the boundaries of AI reasoning and problem-solving. These models demonstrate improved capabilities in understanding, generating language, and autonomous reasoning clos